# Wiki rua (2) tutorial Python notebook

Use this notebook to help you code and calculate answers to the Week 2 tutorial questions. We'll assume you're somewhat familar with Python operations by now, so we'll skip our intro from last week and go straight into the tutorial. A quick reminder about running code in a Jupyter notebook:

> To run notebook cells, click inside the block (add to the code or write your own) and press <code>Ctrl+Enter</code>

Blocks will add themselves to the notebook automatically, or you can use the '+' button on the top ribbon to add more. You can save your notebook and outputs when you're finished.

You can refresh your memory of useful equations from your notes, or from the lecture summary notes on Learn.

Feel free to break up your code into blocks and test them as you go along (e.g., using <code>print()</code> statements to check an intermediate answer) -- this is a great advantage of working in a notebook!


In [60]:
# Load packages - investigate any other packages you might need for this tutorial
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math

In [26]:
# Begin code! 
#known values 
cap = 5 #MW
c_p = 0.48
z = 124 #m
r = 48 #m
# v_cutin = 2 m/s; v_cutout = 25 m/s; v_rated = 15 m/s
wind_bins = list(np.arange(2,26,1))
elev = 200 #m above SL
v_avg = 8 #m/s
T_avg = 15 + 273 #K

In [39]:
# Define rayleigh cdf
def rayleigh(v,v_bar):
    prob = 1- np.exp(-(math.pi/4)*(v/v_bar)**2)
    return prob

# set up values for the Rayleigh distribution
rayleigh_bins = [1.5] #initialise with the lower bound
for i in range(0,len(wind_bins)):
    rayleigh_bins.append(wind_bins[i] + 0.5)

In [52]:
# Calculate frequencies and hours in each bin based on rayleigh dist
freq = []
hours = []
for v in rayleigh_bins:
    f = rayleigh(v+1, v_avg) - rayleigh(v, v_avg)
    freq.append(f)
    hours.append(f*8760)
    

In [48]:
# Calculate power curve
# first, get density at elevation + hub height
P = 101.29 - 0.011837*(elev+z) + (4.793*10**(-7))*(elev+z)**2 #kPa
rho = (P*1000*28.97/1000)/(8.314*T_avg) #kg/m3

# next, calculate power for cut in to rated wind speed, for v > 14 power is rated power = 5 MW
power = []
for v in wind_bins:
    if v < 14:
        p_calc = 0.5*c_p*rho*math.pi*(r**2)*(v**3)/1000 #kW
        power.append(p_calc)
    else:
        power.append(5000) #kW

In [62]:
# calculate annual energy for one turbine and annual CF
energy = []
for i in range(0,len(wind_bins)):
    gen = hours[i] * power[i]
    energy.append(gen)

total_gen = np.sum(energy)

CF = total_gen/(cap*1000*8760)

print('The total generation for the wind turbine is {E:.2f} GWh'.format(E=total_gen/1000000))
print('Capacity factor is {cf:.2f}'.format(cf=CF))

The total generation for the wind turbine is 13.73 GWh
Capacity factor is 0.31


In [68]:
# For three rows of six turbines with 78% losses, calculate gen
farm_gen = total_gen*6 + total_gen*6*0.78 + total_gen*6*0.78*0.78 #kWh

wind_gen_nz = 8.3/(60*60)*1000000 #convert PJ to GWh

pct_nz_wind = (farm_gen/1000000)/wind_gen_nz

print('The total generation for the wind farm is {E:.2f} GWh'.format(E=farm_gen/1000000))
print('The proposed farm is {pct:.1%} of NZ wind generation'.format(pct=pct_nz_wind))

The total generation for the wind farm is 196.76 GWh
The proposed farm is 8.5% of NZ wind generation
